## Exercício 4


In [34]:
'''
Exercicio 4
Edgar Kenji Tanaka
RA 023577
MO444 - Aprendizado de Maquina
Prof. Jacques Wainer
1s. 2017
Data de entrega 15/5/2017

Use os dados do arquivo abalone FaÃ§a os pre-processamentos do exercicio 3.

Usando um 5-fold externo para calcular a accuracia, 
e um 3-fold interno para a escolha dos hyperparametros, 
determine qual algoritimo entre kNN, SVM com kernel RBF, 
redes neurais, Random Forest, e Gradient Boosting Machine tem a maior acuracia. 
Imprima a acuracia com 3 digitos.

Voce nao precisam fazer os loops da validacao cruzada explicitamente. Pode usar a funÃ§Ã£o GridSearchCV do SKlearn..
Reporte a acuracia (com 3 digitos) de cada algoritmo calculada pelo 5-fold CV externo.
Para o algoritmo com maior accuracia, reporte o valor dos hiperparamtertos obtidos para gerar o classificador final.
'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, preprocessing, svm
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing
from sklearn.decomposition import PCA

# Pre-processing
def get_data():
    df = pd.read_csv('abalone.csv', header=None)
    
    # convert feature from numerical to categorical
    y = np.where(df.ix[:,8] > 13, 1,0)

    # drop target column
    x = df.drop(df.columns[[8]], axis=1)

    # one hot encode gender feature
    x = pd.get_dummies(x)    
    
    # standardization 
    scaler = preprocessing.StandardScaler()
    x = scaler.fit_transform(x)
    
    return x, y

# create k-fold of outer loop
X, y = get_data()
outter = StratifiedKFold(n_splits=5, random_state=1)

In [35]:
# 1. knn
# Para o kNN, faca um PCA que mantem 90% da variancia. 
# Busque os valores do k entre os valores 1, 5, 11, 15, 21, 25.
from sklearn.neighbors import KNeighborsClassifier

accs = []
for train_index, test_index in outter.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # PCA
    pca = PCA(n_components=0.9)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    print('sum of variance:',sum(pca.explained_variance_ratio_))
    clf = KNeighborsClassifier(n_neighbors=6)
    
    # find best hyperparams
    parameters = {'n_neighbors':(1, 5, 11, 15, 21, 25)}
    clf = GridSearchCV(KNeighborsClassifier(), parameters, cv=3)
    clf.fit(X_train_pca, y_train)
    
    # train with the best hyperparams
    print('best hyper parameters')
    print(clf.best_params_)
    clf = KNeighborsClassifier(n_neighbors=clf.best_params_['n_neighbors'])
    clf.fit(X_train_pca, y_train)
    accs.append(clf.score(X_test_pca, y_test))
    
    print('----------------------------')

print('accuracies:', accs)
print('mean accuracy:', "%.3f" % np.mean(accs))

sum of variance: 0.932331370613
best hyper parameters
{'n_neighbors': 21}
----------------------------
sum of variance: 0.934819705714
best hyper parameters
{'n_neighbors': 25}
----------------------------
sum of variance: 0.947255388074
best hyper parameters
{'n_neighbors': 21}
----------------------------
sum of variance: 0.933587142903
best hyper parameters
{'n_neighbors': 21}
----------------------------
sum of variance: 0.932053657906
best hyper parameters
{'n_neighbors': 21}
----------------------------
accuracies: [0.88157894736842102, 0.88277511961722488, 0.88263473053892216, 0.88023952095808389, 0.88263473053892216]
mean accuracy: 0.882


In [36]:
# 4. rf
# Para o RF, teste max_features = 2, 3, 5, 7 e n_estimators = 100, 200, 400 e 800.
accs = []
for train_index, test_index in outter.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # find best hyperparams
    parameters = {'max_features':(2, 3, 5, 7), 'n_estimators':(100, 200, 400, 800)}
    clf = GridSearchCV(RandomForestClassifier(random_state=1), parameters, cv=3)
    clf.fit(X_train, y_train)
    
    # train with the best hyperparams
    print('best hyper parameters')
    print(clf.best_params_)
    clf = RandomForestClassifier(random_state=1, 
                                 n_estimators=clf.best_params_['n_estimators'], 
                                 max_features=clf.best_params_['max_features'])
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    accs.append(acc)
    
    print('----------------------------')

print(accs)
print('mean accuracy:', "%.3f" % np.mean(accs))

best hyper parameters
{'max_features': 2, 'n_estimators': 200}
----------------------------
best hyper parameters
{'max_features': 5, 'n_estimators': 800}
----------------------------
best hyper parameters
{'max_features': 2, 'n_estimators': 200}
----------------------------
best hyper parameters
{'max_features': 2, 'n_estimators': 400}
----------------------------
best hyper parameters
{'max_features': 3, 'n_estimators': 800}
----------------------------
[0.87799043062200954, 0.89593301435406703, 0.9017964071856287, 0.88502994011976044, 0.89101796407185629]
mean accuracy: 0.890


In [37]:
# 5. gbm
# Para o GBM (ou XGB) teste para numero de arvores = 30, 70, e 100, 
# com learning rate de 0.1 e 0.05, e profundidade da arvore=5.
# Voce pode tanto usar a versao do SKlearn ou o XGBoost.
accs = []
for train_index, test_index in outter.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # find best hyperparams
    parameters = {'n_estimators':(30, 70, 100), 'learning_rate':(0.1, 0.05)}
    clf = GridSearchCV(GradientBoostingClassifier(max_depth=5, random_state=1), parameters, cv=3)
    clf.fit(X_train, y_train)
    
    # train with the best hyperparams
    print('best hyper parameters')
    print(clf.best_params_)
    clf = GradientBoostingClassifier(max_depth=5, random_state=1, 
                                     n_estimators=clf.best_params_['n_estimators'], 
                                     learning_rate=clf.best_params_['learning_rate'])
    clf.fit(X_train, y_train)
    accs.append(clf.score(X_test, y_test))
    
    print('----------------------------')

print(accs)
print('mean accuracy:', "%.3f" % np.mean(accs))

best hyper parameters
{'learning_rate': 0.1, 'n_estimators': 70}
----------------------------
best hyper parameters
{'learning_rate': 0.1, 'n_estimators': 70}
----------------------------
best hyper parameters
{'learning_rate': 0.05, 'n_estimators': 70}
----------------------------
best hyper parameters
{'learning_rate': 0.05, 'n_estimators': 100}
----------------------------
best hyper parameters
{'learning_rate': 0.05, 'n_estimators': 70}
----------------------------
[0.87918660287081341, 0.89234449760765555, 0.90059880239520962, 0.88742514970059883, 0.89341317365269457]
mean accuracy: 0.891


In [38]:
# 2. svm
# Para o SVM RBF teste para C=2**(-5), 2**(0), 2**(5), 2**(10) 
# e gamma= 2**(-15) 2**(-10) 2**(-5) 2**(0) 2**(5).
accs = []
for train_index, test_index in outter.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # find best hyperparams
    gammas = 2 ** np.array([-15.0, -10.0, -5.0, 0.0, 5.0])
    costs = 2 ** np.array([-5.0, 0.0, 5.0, 10.0])
    parameters = {'C': costs, 'gamma': gammas}
    clf = GridSearchCV(svm.SVC(kernel='rbf'), parameters, cv=3)
    clf.fit(X_train, y_train)
    
    # train with the best hyperparams
    print('best hyper parameters')
    print(clf.best_params_)
    svr = svm.SVC(C=clf.best_params_['C'], gamma=clf.best_params_['gamma'])
    svr.fit(X_train, y_train)
    accs.append(svr.score(X_test, y_test))
    
    print('----------------------------')

print(accs)
print('mean accuracy:', "%.3f" % np.mean(accs))

best hyper parameters
{'C': 1024.0, 'gamma': 0.0009765625}
----------------------------
best hyper parameters
{'C': 32.0, 'gamma': 0.03125}
----------------------------
best hyper parameters
{'C': 1024.0, 'gamma': 0.0009765625}
----------------------------
best hyper parameters
{'C': 1024.0, 'gamma': 0.0009765625}
----------------------------
best hyper parameters
{'C': 1024.0, 'gamma': 0.0009765625}
----------------------------
[0.89832535885167464, 0.88516746411483249, 0.89820359281437123, 0.89341317365269457, 0.88502994011976044]
mean accuracy: 0.892


In [39]:
# 3. rede neural
# Para a rede neural, teste com 3, 7, 10, e 20 neuronios na camada escondida.
accs = []
for train_index, test_index in outter.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # find best hyperparams
    parameters = {'hidden_layer_sizes':((3,),(7,),(10,),(20,))}
    clf = GridSearchCV(MLPClassifier(random_state=1), parameters, cv=3)
    clf.fit(X_train, y_train)    
    
    # train with the best hyperparams
    print('best hyper parameters')
    print(clf.best_params_)
    model = MLPClassifier(random_state=1, hidden_layer_sizes=clf.best_params_['hidden_layer_sizes'])
    model.fit(X_train, y_train)
    acc = model.score(X_test, y_test)
    accs.append(acc)
    
    print('----------------------------')

print(accs)
print('mean accuracy:', "%.3f" % np.mean(accs))

/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't

best hyper parameters
{'hidden_layer_sizes': (7,)}
----------------------------


/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't

best hyper parameters
{'hidden_layer_sizes': (20,)}
----------------------------


/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't

best hyper parameters
{'hidden_layer_sizes': (10,)}
----------------------------


/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't

best hyper parameters
{'hidden_layer_sizes': (10,)}
----------------------------


/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't

best hyper parameters
{'hidden_layer_sizes': (20,)}
----------------------------
[0.88995215311004783, 0.8935406698564593, 0.90898203592814375, 0.89580838323353296, 0.89221556886227549]
mean accuracy: 0.896


## Melhor classificador
Resultados:
- KNN: 0.882
- RF: 0.890
- SVM: 0.892
- GBM: 0.891
- NN: 0.896

O melhor classificador encontrado foi o Neural Network com acurácia de 0.896.<BR>

Abaixo, treinamos este classificador agora com todos os dados do abalone (sem dividir entre treino e teste) a fim de encontrar o hiper parametro ideal.

In [40]:
# Final classifier now with all the data (no split of train and test)
X, y = get_data()

# find best hyperparams
parameters = {'hidden_layer_sizes':((3,),(7,),(10,),(20,))}
clf = GridSearchCV(MLPClassifier(random_state=1), parameters, cv=3)
clf.fit(X, y) 

# train with the best hyperparams
print('best hyper parameters')
print(clf.best_params_)
model = MLPClassifier(random_state=1, hidden_layer_sizes=clf.best_params_['hidden_layer_sizes'])
final_model = model.fit(X, y) # this is the final model to go to production


/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/tanakaed/anaconda3/envs/condaenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't

best hyper parameters
{'hidden_layer_sizes': (7,)}
